Transformación de Datos
Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).
Agrupación y Agregación
Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.
Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.
Documentación
Comentarios claros: Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.
Código legible: Asegúrate de que el código sea legible y esté bien comentado.

In [5]:
import os
import pandas as pd

# Función para cargar los datos del archivo CSV utilizando pandas
def cargar_datos(ruta_archivo):
    # Cargar los datos del archivo CSV utilizando pandas
    datos = pd.read_csv(ruta_archivo)
    return datos

if __name__ == "__main__":
    # Ruta del archivo CSV
    ruta_archivo = '../retail_sales_dataset.csv'
    
    # Verificar si el archivo existe antes de intentar cargarlo
    if os.path.exists(ruta_archivo):
        # Cargar los datos
        df = cargar_datos(ruta_archivo)
        print(df.head())  # Mostrar las primeras filas del DataFrame
    else:
        print(f"El archivo {"../retail_sales_dataset.csv"} no existe.")



   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  


In [6]:

df['Date'] = pd.to_datetime(df['Date'])
df.info()
df.isnull().sum()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    1000 non-null   int64         
 1   Date              1000 non-null   datetime64[ns]
 2   Customer ID       1000 non-null   object        
 3   Gender            1000 non-null   object        
 4   Age               1000 non-null   int64         
 5   Product Category  1000 non-null   object        
 6   Quantity          1000 non-null   int64         
 7   Price per Unit    1000 non-null   int64         
 8   Total Amount      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 70.4+ KB


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [7]:
# Calcular el 'Cost per Unit' restando un 15% del 'Price per Unit'
df['Cost per Unit'] = df['Price per Unit'] * (1 - 0.15)
df.drop('Average Price per Unit', axis=1, inplace=True, errors='ignore')

#Ordenando las columnas
nuevo_orden = ['Transaction ID',"Date",'Customer ID',"Gender", "Age", "Product Category", "Quantity", "Cost per Unit", 'Price per Unit', 'Total Amount']
df = df.reindex(columns=nuevo_orden)

#Creando una nueva columna para el analisis. (profit margin)
df['profit margin'] =df['Price per Unit'] - df["Cost per Unit"]


# Normalizar la columna 'Ventas'
max_value = df['Total Amount'].max()
min_value = df['Total Amount'].min()
df['Total Amount_normalizado'] = df['Total Amount'].apply(lambda x: (x - min_value) / (max_value - min_value))
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Cost per Unit,Price per Unit,Total Amount,profit margin,Total Amount_normalizado
0,1,2023-11-24,CUST001,Male,34,Beauty,3,42.5,50,150,7.5,0.063291
1,2,2023-02-27,CUST002,Female,26,Clothing,2,425.0,500,1000,75.0,0.493671
2,3,2023-01-13,CUST003,Male,50,Electronics,1,25.5,30,30,4.5,0.002532
3,4,2023-05-21,CUST004,Male,37,Clothing,1,425.0,500,500,75.0,0.240506
4,5,2023-05-06,CUST005,Male,30,Beauty,2,42.5,50,100,7.5,0.037975


In [14]:
#Restablecemos el índice para asegurar de que Product Category esté solo como columna y no como índice.
df = df.reset_index(drop=True)

#Clasificación de ventas en categorías significativas (Alta, Media, Baja)

# Definir las categorías según el 'Total Amount'
def clasificar_ventas(total_amount):
    if total_amount >= 500:
        return 'Alta'
    elif total_amount >= 100:
        return 'Media'
    else:
        return 'Baja'

# Crear la nueva columna 'Sales Category' basada en la clasificación
df['Sales Category'] = df['Total Amount'].apply(clasificar_ventas)


# Agrupar los datos por 'Product Category' y 'Gender', y aplicar funciones de agregación
grouped = df.groupby(['Product Category', 'Gender'], as_index=False).agg({
    'Total Amount': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Quantity': ['sum', 'mean']
})

# DataFrame agrupado 'Product Category' y 'Gender'
print("Datos agrupados por 'Product Category' y 'Gender':")
print(grouped)

#Análisis personalizado con apply (Calculando la desviación de cada venta respecto a la media de su grupo)

# Definir una función que calcule la desviación respecto a la media de su grupo
def desviacion_venta(row, mean_by_category):
    categoria = row['Product Category']
    mean_value = mean_by_category[categoria]
    return row['Total Amount'] - mean_value

# Calcular la media por 'Product Category'
mean_by_category = df.groupby('Product Category')['Total Amount'].mean()

# Aplicar la función personalizada al DataFrame
df['Desviación de la media'] = df.apply(lambda row: desviacion_venta(row, mean_by_category), axis=1)

# DataFrame final con la nueva columna 'Desviación de la media'
print("\nDataFrame final con la clasificación de ventas y la desviación de la media:")
print(df)

Datos agrupados por 'Product Category' y 'Gender':
  Product Category  Gender Total Amount                              \
                                    sum        mean count min   max   
0           Beauty  Female        74830  450.783133   166  25  2000   
1           Beauty    Male        68685  487.127660   141  25  2000   
2         Clothing  Female        81275  467.097701   174  25  2000   
3         Clothing    Male        74305  419.802260   177  25  2000   
4      Electronics  Female        76735  451.382353   170  25  2000   
5      Electronics    Male        80170  466.104651   172  25  2000   

                             Quantity            
          std            var      sum      mean  
0  538.735040  290235.443593      418  2.518072  
1  592.899725  351530.083587      353  2.503546  
2  577.016491  332948.030862      441  2.534483  
3  524.116237  274697.829995      453  2.559322  
4  548.644646  301010.947616      439  2.582353  
5  587.129702  344721.287230  